In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import random
random.seed(13)

### Import data

In [ ]:
# maybe use np.where to reduce the data imported

Read

In [ ]:
code = pd.read_csv('data/coding7128.tsv', sep='\t')

In [ ]:
clinical = pd.read_csv('data/tpp_gp_clinical_subset.csv') 

In [ ]:
ic10 = pd.read_csv('data/ic10_lookup.csv')

In [ ]:
GPic10 = pd.read_csv('data/GP_ic10_lookup.csv') 

In [ ]:
scripts = pd.read_csv('data/sample_covid19_tpp_gp_scripts.txt')

have a look at the data

In [ ]:
code.head()
# CTV3 codes (used by GPs, in dataset clinical) and the corresponding disease and incidences
# code can also be measurements, but will not be used by us

In [ ]:
clinical
# GP dataset from every patient
# each line corresponds to one incident with a date (event_dt)
# eid: patient number 
# code_type: ?
# value: measured value of a test or whatever

In [ ]:
ic10
# description of ICD20 codes

In [ ]:
GPic10
#translates CTV3 codes (read_code) to ICD10 (icd10_code)

In [ ]:
scripts
# all medication patients got from their GP, incl. the COVID-19 vaccine 
# needs to be complete if want to compare vaccinated vs. non-vaccinated, but seems like not all vaccinations registered here but just the ones done by the GP

In [ ]:
# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine

# try only to get lines from scripts dataset where dmd_code== a vaccine 
# use function read_csv. only where irgendwas

print('lines in samplescripts: ', len(scripts))
print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

### Preparing the data

#### Adding column with IC10 categories to clinical dataset for simpler analysis later on

In [ ]:
# create a reduced dataset for merging with the clinical dataset 
GPic10_red =  GPic10[['read_code', 'icd10_code']]
# rename to merge ont he ctv3 column with the clinical dataset
GPic10_red = GPic10_red.rename(columns = {'read_code':'code'})
GPic10_red.head()

In [ ]:
clinical_ic10 = pd.merge(clinical, GPic10_red, how='inner', on='code')
clinical_ic10

#### Data Cleaning

In [ ]:
# drop unnecessary columns
clinical_ic10.drop(columns = ['code_type', 'value', 'Unnamed: 0'], inplace = True)
clinical_ic10

In [ ]:
pd.isnull(clinical_ic10).sum()
# no NaN

In [ ]:
pd.isnull(ic10).sum()

In [ ]:
# remove duplicate entries in clinical dataset 
# (for other datasets, it should not be a problem)
clinical_ic10.drop_duplicates(inplace = True) 

# remove NaN in selected column
ic10.dropna(subset=['DESCRIPTION'], inplace = True)

In [ ]:
pd.isnull(ic10).sum()

In [ ]:
# remove not considered clinical events in the clinical dataset (e.g. external factors)

""""
Monica's suggestions: 
remove O-Z (pregnancy, congenital, others, COVID-19, external causes, factors)
maybe keep Q (congenital) as underlying disease, keep all others

Renée:
I would also keep R in there: "In general, categories in this chapter include the less well-defined conditions and symptoms that, without the necessary study of the case to establish a final diagnosis, point perhaps equally to two or more diseases or to two or more systems of the body.

Decision:
O-Z remove, but leave R as adverse event & Q as underlying disease.

"""

### Find who is vaccinated

The meaning of the SNOMED codes ('dmd_code' in the 'scripts' dataset, where medications that the patients got are listed) were found on: https://services.nhsbsa.nhs.uk/dmd-browser/amp/view/159421?ref=YW1wTmFtZT1jb3ZpZCZzZWFyY2hUeXBlPUFNUCZzaG93SW52YWxpZEl0ZW1zPWZhbHNl#ampCodes

Each vaccine has a specific code.

In [ ]:
# broadcasting to find individuals with AstraZEneca vaccine
mask_AZ = scripts['dmd_code']== 39114911000001105
scripts[mask_AZ].head()

In [ ]:
# getting IDs of individuals with any corona vaccine first (compare vaccinates vs. not)
# may look at effects of specific vaccines later
dict_vac = {39326911000001101: 'Moderna', 39230211000001104: 'Janssen', \
            39826711000001101:'Medicago', 39473011000001103: 'Baxter', \
            39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',\
            39373511000001104: 'Valneva'}

for dmd_code in dict_vac:
    mask_vac = scripts['dmd_code']== dmd_code
    print(dict_vac[dmd_code], ':',len(scripts[mask_vac]))

In [ ]:
# mask for any vaccine
mask_vac = scripts['dmd_code'].isin(dict_vac)

# number of vaccinated people
print('Number of vaccinated people:', len(scripts[mask_vac]))

In [ ]:
# finding eids of vaccinated people in the dataset
id_vac = scripts[mask_vac]['eid'] 
id_vac

In [ ]:
# finding events of the vaccinated people, but only events before vaccination found 
# -> may need a more recent dataset
# need to separate clinical events before & after vaccine
mask_vac_clin = clinical_ic10['eid'].isin(id_vac)
clinical_ic10[mask_vac_clin] 

# categorize event as before or after COVID-19 vaccination 

### make 'event_date' in 'clinical' be interpreted as a date

In [ ]:
clinical_ic10.head(10)

In [ ]:
from datetime import datetime, timedelta

### only to understand the function, can be deleted later

In [ ]:
clinical_ic10['event_dt'].unique()
# find out if the other dates are e.g. 06/01/2020 or 6/1/2020

In [ ]:
example_date = clinical_ic10['event_dt'].unique()[0]

In [ ]:
example_date_new = datetime.strptime(example_date, '%d/%m/%Y')
example_date_new #2020

In [ ]:
example_date_2 = clinical_ic10['event_dt'].unique()[1]
example_date_2 

In [ ]:
example_date_2_new = datetime.strptime(example_date_2, '%d/%m/%Y')
example_date_2_new #2019

In [ ]:
# compare the 2 dates
example_date_new > example_date_2_new # 2020 > 2019 --> True
# recent > older --> True

### actually convert the column into the before/after

In [ ]:
# convert entire event_dt in datetime format
clinical_ic10['event_dt_date_format'] = pd.to_datetime(clinical_ic10['event_dt'])
clinical_ic10

In [ ]:
# only to understand the function, can be deleted later!!
# compare 2 dates (first row and last row) from the new column 

print(clinical_ic10['event_dt_date_format'].iloc[-1]) # 2019
print(clinical_ic10['event_dt_date_format'].iloc[0]) # 2020
clinical_ic10['event_dt_date_format'].iloc[0] > clinical_ic10['event_dt_date_format'].iloc[-1]

### include unvaccinated people (needed for further calculations)
for including them in the dataframe to do:
* event_dt = 0 or NaN (for selecting them)
* get random date of vaccinations for each unvaccinated person -> put it as the issue date in unvaccinated people
* using this average date, define "before/after" --> this will be used to classify medical events as "underlying disease" or "adverse event" for before and after respectively.

alternatively, just make another subset, again make
* get random date of vaccinations (average issue_date) for each unvaccinated person -> put it as the issue date in unvaccinated people
* using this average date, define "underlying disease/adverse event" like "before/after" by comparing to the average date


### subset 'scripts' to only those patients (eid) that got the COVID-19 vaccine and drop 'Unnamed: 0' column

In [ ]:
scripts.drop(columns = ['Unnamed: 0'], inplace = True)
mask_scripts_vac = scripts['dmd_code'].isin(dict_vac)
scripts_vaccines_only = scripts[mask_scripts_vac] 

len(scripts[mask_scripts_vac]) # corresponds to finding above: 7 pfizer and 23 astrazeneca

In [ ]:
scripts_vaccines_only.head() # contains eid and date of vaccination
# length: 30

In [ ]:
clinical_ic10.head()
# length: 4156
len(clinical_ic10['eid'].unique()) # some eid are in there multiple times (of course)

### add a column with the date of vaccination to clinical_ic10

In [ ]:
# merge clinical_ic10 and scripts_vaccines_only to add the issue_date (date of vaccine) to the
# Monica's suggestion: merge 'outer' to also include people that got vaccinated but have no medical events noted
clinical_vac = pd.merge(clinical_ic10, scripts_vaccines_only, how ='outer', on ='eid')

In [ ]:
clinical_vac

In [ ]:
len(clinical_vac) 
# total amounts of events: vaccinations and medical events from clinical_ic10

In [ ]:
clinical_vac['issue_date'].isna().sum()
# 4150 events of unvaccinated, 34 of vaccinated patients

### add column in clinical_ic10 that says whether a patient is vaccinated or not

In [ ]:
mask_vaccinated = clinical_full['eid'].isin(id_vac)
clinical_vac['vaccinated'] = mask_vaccinated

# check how many vaccinated people there are
clinical_vac['vaccinated'].value_counts()
# 34 -> correct

# clinical_ic10['eid'].loc[clinical_ic10['vaccinated']==True]

In [ ]:
clinical_vac
# rows with event_dt=NaN: vaccinated eids with medical events
# maybe rename it

### add random issue date to events of unvaccinated patients to 'issue_date'  in clinical_vac 

In [ ]:
# make list from 'issue_date' in scripts_vaccines_only
issue_dates = scripts_vaccines_only['issue_date'].tolist()
issue_dates

In [ ]:
random.choice(issue_dates)

In [ ]:
# randomly sample a date from issue_dates for each unvaccinated patient

for i in range (len(clinical_vac['eid'])):
    if clinical_vac['vaccinated'].loc[i] == False:
        clinical_vac['issue_date'].loc[i] = random.choice(issue_dates) # we just want the values attribute, not in an array
        
clinical_vac

### add a column where it says if the event (event_dt_date_format) was before or after the vaccination (issue_date)

In [ ]:
# monica: is this step really needed? seems like issue_date is in date format already.
clinical_vac['issue_date_format'] = pd.to_datetime(clinical_vac['issue_date'])
clinical_vac
# renée: I guess it's safer, since the dtype is 'datetime64' in the new column and 'object' in the old one:
print(clinical_vac['issue_date']) 
print(clinical_vac['issue_date_format'])

In [ ]:
clinical_vac['event_dt_date_format'].iloc[0]>=clinical_vac['issue_date_format'].iloc[0]

In [ ]:
before_after_vaccine = []
for i in range (len(clinical_vac)):
    if clinical_vac['event_dt_date_format'].iloc[i] >= clinical_vac['issue_date_format'].iloc[i]:
        # edit from monica: fill in the 'before/after' into the list, not overwrite the variable 
        before_after_vaccine.append('after')
    else:
        before_after_vaccine.append('before')
clinical_vac['before_after_vaccine'] = before_after_vaccine

In [ ]:
clinical_vac

In [ ]:
clinical_vac['before_after_vaccine'].value_counts() # no after -> correct

# Analysing clinical events

IC10 codes are structured. The following website was used for looking up the meanings of the categories (e.g. respiratory diseases):
https://www.icd10data.com/ICD10CM/Codes

In [ ]:
ic10.head()

the GP_ic10 has the read code which is the CTV3. 
(However, when things get nasty, there are multiple ICD10 codes per CTV3 codes and viceversa… there is a mapping status which tells you whether the code is primary, duplicated, to revision, etc)

In [ ]:
# WILL PROBABLY NOT USE THIS

# just to see if corresponding CTV3 to IC10 works

# broadcasting for vaccinated comorbidities(CTV3) in ICD10
mask_vac_ic10 = GPic10['read_code'].isin(clinical_ic10[mask_vac_clin]['code']) 
GPic10[mask_vac_ic10]

# mask_vac: people that got any vaccine
# search for patients that got the vaccine in GPic10
# and look what clinical events they had via the read_code (e.g. XE01qn)


if time: analyse specific diseases such as myocarditis etc.

In [ ]:
clinical_ic10

In [ ]:
ic10.head(3)

In [ ]:
# mask for getting codes related to myocarditis
mask_myocarditis = ic10['DESCRIPTION'].str.contains('myocarditis', case = False)
ic10[mask_myocarditis]

In [ ]:
# getting data with myocarditis

# the alternative code 'ALT_CODE' used here, as this was used in the lookup table GPic10
myocarditis = ic10[mask_myocarditis]['ALT_CODE']

clinical_ic10[clinical_ic10['icd10_code'].isin(myocarditis)]
# nobody had myocarditis in this restricted data

In [ ]:
## ishaemic heart disease
mask_ihd = ic10['DESCRIPTION'].str.contains('ischaemic heart disease', case = False)
ischaemicHD = ic10[mask_ihd]['ALT_CODE']
clinical_ic10[clinical_ic10['icd10_code'].isin(ischaemicHD)]

several comorbidities

In [ ]:
# getting people with specific diseases

# new column for filling with selected disease
clinical_ic10['selected disease'] = np.nan 

# list of specific diseases, to be decided on
diseases_selec = ['ischaemic heart disease', 'cardiomyopathy', 'heart failure',\
                  'myocarditis'] 

for disease in diseases_selec:
    
    # broadcasting for selected disease
    mask = ic10['DESCRIPTION'].str.contains(disease, case = False) 
    # broadcasting for selected disease
    sub_disease = clinical_ic10[clinical_ic10['icd10_code'].isin(ic10[mask]['ALT_CODE'])]
    
    for i in sub_disease.index:
        # fill dataset with selected disease label
        clinical_ic10.loc[i,'selected disease'] = disease 

In [ ]:
# counting how many people in the dataset have a specific disease
clinical_ic10['selected disease'].value_counts()

# getting people w. different categories of underlying diseases

# Question 1: 
## 1. How common are "adverse events"? (i.e. how many percent of vaccinated people have registered medical events after the vaccine) compare vaccinated & nonvaccinated
## 2. What about people with medical history?
## Are they more common in people with (specific) underlying medical conditions? i.e.: 
* plot percentage of AEs with & without underlying conditions
* plot percentages for specific underlying conditions (not considering if several conditions at once, plot for each category then), for vaccinated and non-vaccinated people
* (optional: just for vaccinated people: plot what types of AEs (in IC10 categories) are common.

In [ ]:
# code layout for getting the different categories

# subsetting into categories using IC10
# e.g. I00-I99 Diseases of the circulatory system, more details available too, e.g. Ischemic heart diseases
GPic10.dropna(inplace=True) # drop NaN to enable making mask

m_circ = GPic10['icd10_code'].str.startswith('I')
GPic10[m_circ]['read_code']

In [ ]:
# getting corresponding CTV3 codes
m_circ_CTV3 = code['coding'].isin(GPic10[m_circ]['read_code'])
code[m_circ_CTV3]

In [ ]:
 # getting subjects with this condition
m_circ_sub = clinical['code'].isin(code[m_circ_CTV3]['coding']) # another mask

# correspond to vaccinated or not

In [ ]:
# no one with any vaccine had circulatory issue before vaccine on this limited dataset
scripts[mask_vac]['eid'].isin(clinical[m_circ_sub]['eid']).sum()